In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook
from joblib import Parallel, delayed
from functools import reduce
#from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.set_option('display.max_columns', None)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.metrics import log_loss , accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [14]:
df=pd.read_csv("/content/Final_df.csv")

###Encoding

In [15]:
le = preprocessing.LabelEncoder()

df["Game_ID"] = le.fit_transform(df["Game_ID"])
game_mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [16]:
df["Home Team"] = le.fit_transform(df["Home Team"])

mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [17]:
df["Away Team"] = df["Away Team"].apply(lambda x: mapping[x])

In [18]:
Train=df[df['Train_y']>0]
Train.drop("Train_y", axis=1, inplace=True)

In [19]:
Test=df[df['Train_y']==0]
Test.drop(['Score','Train_y'],axis=1,inplace=True)

In [20]:
# to avoid nans, will map score onli in the training set
Train["Score"] = le.fit_transform(Train["Score"])
score_mapping = dict(zip(le.classes_, range(len(le.classes_))))

In [21]:
Train

,Game_ID,Shots,SoT,Accurate passes,Inaccurate passes,Passes,SoT/shots,acc_pass,adv SoT/shots,adv acc_pass,Date,Season_y,Match_ID_y,Home Team,Away Team,Score,created_at,year,month,day,weekday,weekofyear
0,0,19.0,7.0,327.0,83.0,410.0,0.368421,0.797561,0.000000,0.753582,2016-11-08,1,42.0,3,14,0,2016-11-08,2016.0,11,8,1,45
1,1,18.0,6.0,259.0,71.0,330.0,0.333333,0.784848,0.250000,0.868284,2016-11-06,1,30.0,2,13,1,2016-11-06,2016.0,11,6,6,44
3,3,6.0,4.0,394.0,87.0,481.0,0.666667,0.819127,0.222222,0.810757,2017-11-21,2,24.0,0,5,1,2017-11-21,2017.0,11,21,1,47
6,6,11.0,2.0,448.0,82.0,530.0,0.181818,0.845283,0.375000,0.830189,2018-05-04,2,186.0,20,5,2,2018-05-04,2018.0,5,4,4,18
9,9,10.0,4.0,762.0,193.0,955.0,0.400000,0.797906,0.285714,0.807927,2018-01-23,2,84.0,8,9,0,2018-01-23,2018.0,1,23,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,641,7.0,5.0,324.0,94.0,418.0,0.714286,0.775120,0.400000,0.714706,2017-02-19,1,36.0,3,17,1,2017-02-19,2017.0,2,19,6,7
642,642,10.0,3.0,279.0,77.0,356.0,0.300000,0.783708,0.272727,0.836364,2018-02-20,2,116.0,2,7,2,2018-02-20,2018.0,2,20,1,8
643,643,11.0,2.0,352.0,93.0,445.0,0.181818,0.791011,0.333333,0.786177,2016-08-14,1,130.0,10,3,0,2016-08-14,2016.0,8,14,6,32
644,644,9.0,4.0,455.0,82.0,537.0,0.444444,0.847300,0.600000,0.802198,2018-01-30,2,103.0,7,18,0,2018-01-30,2018.0,1,30,1,5


###Creating Folds based on time

In [22]:
import datetime as dt

In [23]:
validation_split1 = dt.datetime.strptime(
    '2018-08-21 00:00:00', '%Y-%m-%d %H:%M:%S') - dt.timedelta(days=220)
print('fold1 train until:', validation_split1)
validation_split2 = validation_split1 - dt.timedelta(days=298)
print('fold2 train until:', validation_split2)

fold1 train until: 2018-01-13 00:00:00
fold2 train until: 2017-03-21 00:00:00


In [25]:
Train['created_at'] = pd.to_datetime(df['Date'])

In [26]:
fold1= Train[Train['created_at'] <= validation_split2].reset_index(drop=True)
fold2= Train[Train['created_at'] > validation_split2 ][ Train['created_at'] <= validation_split1 ].reset_index(drop=True)
fold3=Train[Train['created_at'] > validation_split1].reset_index(drop=True)

In [27]:
print(fold1.shape)
print(fold2.shape)
print()
print(fold3.shape)


(166, 22)
(109, 22)

(137, 22)


In [28]:
params={
 'learning_rate': 0.008054012237639782,
 'min_child_samples': 77,
 'n_estimators': 3000,
 'num_leaves': 22,
 'objective': 'multiclass',
  'silent':True,
  'num_class': 3
  }
params_1 = {
    'booster': 'gbtree',
    'max_depth': 5, 
    'learning_rate': 0.025,
    'min_split_loss': 0.01,
    'min_child_weight': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.25,
    'alpha': 2,
    'lambda': 3,
    'objective': 'multiclass',
    'num_class': 3}

In [35]:

fold1.fillna(0,inplace=True)
fold2.fillna(0,inplace=True)
fold3.fillna(0,inplace=True)
Test.fillna(0,inplace=True)
# Lists to store predictions and losses
predictions = []
losses = []
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
X=pd.concat([fold1,fold2])
model.fit(X.drop(['created_at','weekday','weekofyear','Score','Date'],axis=1),X['Score'])
preds =model.predict_proba((Test.drop(['created_at','weekday','weekofyear','Date'],axis=1)))
predictions.append(preds)


model=GaussianNB()
X=pd.concat([fold1,fold3])
model.fit(X.drop(['created_at','weekday','weekofyear','Score','Date'],axis=1),X['Score'])
preds =model.predict_proba((Test.drop(['created_at','weekday','weekofyear','Date'],axis=1)))
predictions.append(preds)

model=GaussianNB()
X=pd.concat([fold2,fold3])
model.fit(X.drop(['created_at','weekday','weekofyear','Score','Date'],axis=1),X['Score'])
preds =model.predict_proba((Test.drop(['created_at','weekday','weekofyear','Date'],axis=1)))
predictions.append(preds)



In [36]:

fold1.fillna(0,inplace=True)
fold2.fillna(0,inplace=True)
fold3.fillna(0,inplace=True)
Test.fillna(0,inplace=True)
# Lists to store predictions and losses
losses = []
from lightgbm import LGBMClassifier
model=LGBMClassifier(**params_1)
X=pd.concat([fold1,fold2])
model.fit(X.drop(['created_at','weekday','weekofyear','Score','Date'],axis=1),X['Score'])
preds =model.predict_proba((Test.drop(['created_at','weekday','weekofyear','Date'],axis=1)))
predictions.append(preds)


model=LGBMClassifier(**params)
X=pd.concat([fold1,fold3])
model.fit(X.drop(['created_at','weekday','weekofyear','Score','Date'],axis=1),X['Score'])
preds =model.predict_proba((Test.drop(['created_at','weekday','weekofyear','Date'],axis=1)))
predictions.append(preds)

model=LGBMClassifier(**params)
X=pd.concat([fold2,fold3])
model.fit(X.drop(['created_at','weekday','weekofyear','Score','Date'],axis=1),X['Score'])
preds =model.predict_proba((Test.drop(['created_at','weekday','weekofyear','Date'],axis=1)))
predictions.append(preds)

In [37]:
predictions =np.mean(predictions,0)

In [38]:
pd.DataFrame(predictions)

,0,1,2
0,0.228085,0.374370,0.397546
1,0.129111,0.523265,0.347623
2,0.134674,0.535306,0.330020
3,0.144952,0.604778,0.250269
4,0.178642,0.571885,0.249472
...,...,...,...
229,0.394062,0.341969,0.263970
230,0.163860,0.598770,0.237371
231,0.148333,0.525138,0.326529
232,0.140657,0.548058,0.311285


In [39]:
t=Test.copy()
game_mapping = {value:key for key, value in game_mapping.items()}
t["Game_ID"]=t["Game_ID"].apply(lambda x: game_mapping[x])

,Game_ID,Shots,SoT,Accurate passes,Inaccurate passes,Passes,SoT/shots,acc_pass,adv SoT/shots,adv acc_pass,Date,Season_y,Match_ID_y,Home Team,Away Team,created_at,year,month,day,weekday,weekofyear
2,ID_03VRW8WV,9.0,3.0,371.0,88.0,459.0,0.333333,0.808279,0.600000,0.813953,2018-11-28,3,87.0,8,1,2018-11-28,2018.0,11,28,2,48
4,ID_05YW7A1R,8.0,2.0,331.0,78.0,409.0,0.250000,0.809291,0.166667,0.815842,2019-04-23,3,190.0,18,8,2019-04-23,2019.0,4,23,1,17
5,ID_064HV3UA,10.0,4.0,352.0,72.0,424.0,0.400000,0.830189,0.363636,0.860849,2018-10-27,3,59.0,11,15,2018-10-27,2018.0,10,27,5,43
7,ID_06SV1XSM,8.0,2.0,513.0,77.0,590.0,0.250000,0.869492,0.000000,0.771084,2018-10-28,3,60.0,9,21,2018-10-28,2018.0,10,28,6,43
8,ID_08N8KYTM,9.0,2.0,213.0,57.0,270.0,0.222222,0.788889,0.368421,0.884226,2018-10-16,3,46.0,12,4,2018-10-16,2018.0,10,16,1,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,ID_ZMO6O1V6,15.0,7.0,619.0,182.0,801.0,0.466667,0.772784,0.277778,0.822633,2018-12-14,3,110.0,19,9,2018-12-14,2018.0,12,14,4,50
637,ID_ZQOB9ZQL,14.0,4.0,333.0,85.0,418.0,0.285714,0.796651,0.333333,0.792135,2019-02-26,3,144.0,12,19,2019-02-26,2019.0,2,26,1,9
638,ID_ZRZC49M1,9.0,3.0,402.0,81.0,483.0,0.333333,0.832298,0.000000,0.830239,2018-12-04,3,96.0,1,21,2018-12-04,2018.0,12,4,1,49
639,ID_ZTDZNU6I,7.0,1.0,344.0,76.0,420.0,0.142857,0.819048,0.000000,0.810959,2019-04-24,3,185.0,21,1,2019-04-24,2019.0,4,24,2,17


In [40]:
sub = pd.DataFrame(predictions)
game_mapping = {value:key for key, value in game_mapping.items()}
sub['Game_ID'] =  list(t["Game_ID"].values)
sub

,0,1,2,Game_ID
0,0.228085,0.374370,0.397546,ID_03VRW8WV
1,0.129111,0.523265,0.347623,ID_05YW7A1R
2,0.134674,0.535306,0.330020,ID_064HV3UA
3,0.144952,0.604778,0.250269,ID_06SV1XSM
4,0.178642,0.571885,0.249472,ID_08N8KYTM
...,...,...,...,...
229,0.394062,0.341969,0.263970,ID_ZMO6O1V6
230,0.163860,0.598770,0.237371,ID_ZQOB9ZQL
231,0.148333,0.525138,0.326529,ID_ZRZC49M1
232,0.140657,0.548058,0.311285,ID_ZTDZNU6I


In [41]:
sub = sub.rename(columns={0:'Away Win',1:'Draw',2:'Home Win'})

In [42]:
L=[]
for x,y,z in zip(sub['Away Win'],sub['Draw'],sub['Home Win']):
  if(x>=0.48):
    L.append('Away Win')
  elif y>=0.485:
    L.append('Draw')
  else:
    L.append('Home Win')
sub['Score']=L

,Away Win,Draw,Home Win,Game_ID,Score
0,0.228085,0.374370,0.397546,ID_03VRW8WV,Home Win
1,0.129111,0.523265,0.347623,ID_05YW7A1R,Draw
2,0.134674,0.535306,0.330020,ID_064HV3UA,Draw
3,0.144952,0.604778,0.250269,ID_06SV1XSM,Draw
4,0.178642,0.571885,0.249472,ID_08N8KYTM,Draw
...,...,...,...,...,...
229,0.394062,0.341969,0.263970,ID_ZMO6O1V6,Home Win
230,0.163860,0.598770,0.237371,ID_ZQOB9ZQL,Draw
231,0.148333,0.525138,0.326529,ID_ZRZC49M1,Draw
232,0.140657,0.548058,0.311285,ID_ZTDZNU6I,Draw


In [43]:
sub=sub[['Game_ID','Score']]
sub

,Game_ID,Score
0,ID_03VRW8WV,Home Win
1,ID_05YW7A1R,Draw
2,ID_064HV3UA,Draw
3,ID_06SV1XSM,Draw
4,ID_08N8KYTM,Draw
...,...,...
229,ID_ZMO6O1V6,Home Win
230,ID_ZQOB9ZQL,Draw
231,ID_ZRZC49M1,Draw
232,ID_ZTDZNU6I,Draw


In [44]:
sub.to_csv('Top9_solution.csv',index=False)